#Hi
hello

code extra

import numpy as np
from sympy import *
from sympy.abc import m, t  # , g
from sympy.physics.mechanics import *
from pydy.viz.shapes import Cylinder, Sphere
from pydy.viz.visualization_frame import VisualizationFrame
from sympy.physics.vector import init_vprinting, vlatex
from pydy.viz.scene import Scene
from scipy.integrate import odeint
from pydy.codegen.ode_function_generators import generate_ode_function
# from sympy.physics.vector import *
from matplotlib import pyplot as plt

init_vprinting(use_latex='mathjax', pretty_print=False)
# const
eng_out = 6
mass = 0.151
inertia_local = (1, 1, 1)
COM_local = (0, 0, 30)
para_d = 6
para_f = 1
g_a = -9.81
fpr = 2

para_local = (0, 0, 50)

eng_lut = {'A': [1.26, 2.50],
           'B': [2.51, 5.00],
           'C': [5.01, 10.0],
           'D': [10.01, 20.0],
           'E': [20.01, 40.0],
           'F': [40.01, 80.0],
           'G': [80.01, 160],
           'H': [160.01, 320]}  # add range force


def vector_base(ve):
    return ve.x, ve.y, ve.z


def set_pos_ro(p, p_l):
    p.set_pos(Eng, dot(p_l, Ro_un))


def vis_f(poi, sh):
    return VisualizationFrame(Ro, poi, sh)


def motor_turn():
    pass


class Engine:
    def __init__(self, parram):
        self.lett = parram.pop(0)
        self.force, self.delay = parram.split('-')

        self.impulse = self.impulse_calc()
        self.shut = self.impulse / self.force

    def impulse_calc(self):
        im_ra = eng_lut[self.lett]
        im_re = im_ra[0] + self.force/10*im_ra[1]-im_ra[0]
        return im_re


# reference frames
N = ReferenceFrame('N')
pad = Point('pad')


# rocket init
Ro = ReferenceFrame('Ro')  # rocket refernce, thrust always at cent, maybe ad mom later
eng_ref = ReferenceFrame('eng_ref')
Ro_un = vector_base(Ro)

Eng = Point('Eng')
COM = Point('COM')
Para = Point('Para')
roc_cent = Point('ro_cent')

set_pos_ro(COM, COM_local)
set_pos_ro(Para, para_local)
set_pos_ro(roc_cent, np.array(para_local)/2)

inert = (inertia(Ro, 1, 1, 1), COM)
rock = RigidBody('Rock', COM, Ro, m, inert)

# point shpere
com_sp = Sphere(color='yellow', radius=0.125)
eng_sp = Sphere(color='black', radius=0.1)
nose_sp = Sphere(color='black', radius=0.1)

# point vis
com_p = vis_f(COM, com_sp)
eng_p = vis_f(Eng, eng_sp)
nose_p = vis_f(Para, nose_sp)


# dim simboys
h = dynamicsymbols('h')
v = dynamicsymbols('h', 1)
a = dynamicsymbols('h', 2)
theta = dynamicsymbols('th')
w = dynamicsymbols('th', 1)
t_mot = dynamicsymbols('t_mot')
force, fp = dynamicsymbols('f f_p')
t_x, ty, tz = dynamicsymbols('T_x T_y T_z')  # todo replace with f*r

# norm sim
p_t, g_t, t_shut = symbols('p_t g_t t_sh')

# forces
g_f = g_a*m*N.y
g = (g_f, COM)

f = Piecewise((-force*eng_ref.z, t <= t_shut), (0, True))  # hofuly this works
f_mot = (f, Eng)
f_p_f = Piecewise((-fpr*Ro.z*fp, p_t < t <= p_t + 1), (0, True))  # hofuly this works, maybe just loop and solve for curr dt
f_para = (f_p_f, Para)

mot_rad = Eng.pos_from(COM)
# torques
T_r = cross(mot_rad, f)

Eng.v2pt_theory(COM, N, Ro)


out_s = {}
engine_size = 'c'


In [1]:
#imports
import numpy as np
from sympy import *
from sympy.abc import m, t  # , g
from sympy.physics.mechanics import *
from pydy.viz.shapes import Cylinder, Sphere
from pydy.viz.visualization_frame import VisualizationFrame
from sympy.physics.vector import init_vprinting, vlatex
from pydy.viz.scene import Scene
from scipy.integrate import odeint
from pydy.codegen.ode_function_generators import generate_ode_function
# from sympy.physics.vector import *
from matplotlib import pyplot as plt

# initialise

In [2]:
# constants
init_vprinting(use_latex='mathjax', pretty_print=False)
# const
eng_out = 6
mass = 0.151
inertia_local = (1, 1, 1)
COM_local = (0, 0, 30)
para_d = 6
para_f = 1
g_a = -9.81
fpr = 2

para_local = (0, 0, 50)

eng_lut = {'A': [1.26, 2.50],
           'B': [2.51, 5.00],
           'C': [5.01, 10.0],
           'D': [10.01, 20.0],
           'E': [20.01, 40.0],
           'F': [40.01, 80.0],
           'G': [80.01, 160],
           'H': [160.01, 320]}  # add range force

In [5]:
# functions
def vector_base(ve):
    return ve.x, ve.y, ve.z


def set_pos_ro(p, p_l):
    p.set_pos(Eng, np.dot(p_l, Ro_un))


def vis_f(poi, sh):
    return VisualizationFrame(Ro, poi, sh)


def motor_turn():
    pass


class Engine:
    def __init__(self, parram):
        self.lett = parram.pop(0)
        self.force, self.delay = parram.split('-')

        self.impulse = self.impulse_calc()
        self.shut = self.impulse / self.force

    def impulse_calc(self):
        im_ra = eng_lut[self.lett]
        im_re = im_ra[0] + self.force/10*im_ra[1]-im_ra[0]
        return im_re

# refence frames


In [6]:

# reference frames
N = ReferenceFrame('N')
pad = Point('pad')


# rocket init
Ro = ReferenceFrame('Ro')  # rocket refernce, thrust always at cent, maybe ad mom later
eng_ref = ReferenceFrame('eng_ref')
Ro_un = vector_base(Ro)

Eng = Point('Eng')
COM = Point('COM')
Para = Point('Para')
roc_cent = Point('ro_cent')

set_pos_ro(COM, COM_local)
set_pos_ro(Para, para_local)
set_pos_ro(roc_cent, np.array(para_local)/2)

inert = (inertia(Ro, 1, 1, 1), COM)
rock = RigidBody('Rock', COM, Ro, m, inert)

Now for objects

In [7]:
# point shpere
com_sp = Sphere(color='yellow', radius=0.125)
eng_sp = Sphere(color='black', radius=0.1)
nose_sp = Sphere(color='black', radius=0.1)

# point vis
com_p = vis_f(COM, com_sp)
eng_p = vis_f(Eng, eng_sp)
nose_p = vis_f(Para, nose_sp)

In [12]:

# dim simboys
h = dynamicsymbols('h')
v = dynamicsymbols('h', 1)
a = dynamicsymbols('h', 2)
theta = dynamicsymbols('th')
w = dynamicsymbols('th', 1)
t_mot = dynamicsymbols('t_mot')
force, fp = dynamicsymbols('f f_p')
t_x, ty, tz = dynamicsymbols('T_x T_y T_z')  # todo replace with f*r

# norm sim
g_t = symbols('g_t')
p_t = para_d
t_shut = 5

In [15]:
# forces
g_f = g_a*m*N.y
g = (g_f, COM)

f = Piecewise((-force, t <= t_shut), (0, True))  # hofuly this works
f_mot = (f*eng_ref.z, Eng)
f_p_f = Piecewise((-fpr*fp, p_t < t), (0, True))  # hofuly this works, maybe just loop and solve for curr dt
f_para = (f_p_f*Ro.z, Para)

mot_rad = Eng.pos_from(COM)
# torques
T_r = np.cross(mot_rad, f)

AxisError: axisa: axis -1 is out of bounds for array of dimension 0

In [16]:
help(generate_ode_function)


Help on function generate_ode_function in module pydy.codegen.ode_function_generators:

generate_ode_function(*args, **kwargs)
    Generates a numerical function which can evaluate the right hand
    side of the first order ordinary differential equations from a
    system described by one of the following three symbolic forms:
    
        [1] x' = F(x, t, r, p)
    
        [2] M(x, p) x' = F(x, t, r, p)
    
        [3] M(q, p) u' = F(q, u, t, r, p)
            q' = G(q, u, t, r, p)
    
    where
    
        x : states, i.e. [q, u]
        t : time
        r : specified (exogenous) inputs
        p : constants
        q : generalized coordinates
        u : generalized speeds
        M : mass matrix (full or minimum)
        F : right hand side (full or minimum)
        G : right hand side of the kinematical differential equations
    
    The generated function is of the form F(x, t, p) or F(x, t, r, p)
    depending on whether the system has specified inputs or not.
    
    Par

In [17]:
eng_st = 'A10-5'
En = Engine(eng_st)

AttributeError: 'str' object has no attribute 'pop'

In [18]:
help(kanes_method)

NameError: name 'kanes_method' is not defined